In [4]:
import numpy as np
import pandas as pd

In [7]:
def haar_dwt(data):
    """
    Performs a single level Discrete Wavelet Transform (DWT) using the Haar wavelet.

    Args:
        data (np.ndarray): A 1D numpy array of the input data.
                           The length of the data should be a power of 2 for simplicity
                           in this implementation. If not, it will be padded with zeros.

    Returns:
        tuple: A tuple containing:
            - approximation_coeffs (np.ndarray): The approximation coefficients (low-pass).
            - detail_coeffs (np.ndarray): The detail coefficients (high-pass).
    """
    n = len(data)
    if n % 2 != 0:
        # Pad with zeros if length is odd for a single level transform
        # For multi-level, usually data is padded to nearest power of 2
        print("Warning: Data length is odd. Padding with a zero for single level Haar DWT.")
        data = np.pad(data, (0, 1), 'constant')
        n = len(data)

    approximation_coeffs = np.zeros(n // 2)
    detail_coeffs = np.zeros(n // 2)

    for i in range(n // 2):
        # Haar basis functions:
        # Sum (average) for approximation: (x[2i] + x[2i+1]) / sqrt(2)
        # Difference for detail: (x[2i] - x[2i+1]) / sqrt(2)
        # We'll omit the 1/sqrt(2) normalization for simplicity in this example,
        # but it's crucial for energy preservation in a true orthonormal transform.
        # If you need perfect reconstruction later, it's good to include it.
        # For practical purposes, often it's absorbed into reconstruction.

        approximation_coeffs[i] = (data[2 * i] + data[2 * i + 1]) / np.sqrt(2)
        detail_coeffs[i] = (data[2 * i] - data[2 * i + 1]) / np.sqrt(2)

    return approximation_coeffs, detail_coeffs

def haar_idwt(approximation_coeffs, detail_coeffs):
    """
    Performs a single level Inverse Discrete Wavelet Transform (IDWT) using the Haar wavelet.

    Args:
        approximation_coeffs (np.ndarray): The approximation coefficients.
        detail_coeffs (np.ndarray): The detail coefficients.

    Returns:
        np.ndarray: The reconstructed 1D data.
    """
    n_approx = len(approximation_coeffs)
    n_detail = len(detail_coeffs)

    if n_approx != n_detail:
        raise ValueError("Approximation and detail coefficients must have the same length.")

    reconstructed_data = np.zeros(n_approx * 2)

    for i in range(n_approx):
        # Inverse Haar transform:
        # x[2i] = (approximation[i] + detail[i]) / sqrt(2)
        # x[2i+1] = (approximation[i] - detail[i]) / sqrt(2)
        # We need to account for the sqrt(2) division done during DWT.
        reconstructed_data[2 * i] = (approximation_coeffs[i] + detail_coeffs[i]) / np.sqrt(2)
        reconstructed_data[2 * i + 1] = (approximation_coeffs[i] - detail_coeffs[i]) / np.sqrt(2)

    return reconstructed_data

def multi_level_haar_dwt(data, levels):
    """
    Performs multi-level Discrete Wavelet Transform (DWT) using the Haar wavelet.

    Args:
        data (np.ndarray): A 1D numpy array of the input data.
        levels (int): The number of decomposition levels.

    Returns:
        list: A list of arrays, where the first element is the final approximation
              coefficients, and subsequent elements are the detail coefficients
              from each level, starting from the deepest.
              e.g., [final_approximation, detail_level_N, detail_level_N-1, ..., detail_level_1]
    """
    coeffs = []
    current_data = np.array(data, dtype=float)  # Ensure float type for calculations

    for level in range(levels):
        n = len(current_data)
        if n < 2:
            print(f"Warning: Data too short for further decomposition at level {level+1}. Stopping.")
            break

        # Pad with zeros if length is not a power of 2 for multi-level DWT
        # This is a common approach to handle arbitrary length data for multi-level DWT
        next_power_of_2 = 2**np.ceil(np.log2(n)).astype(int)
        if n < next_power_of_2:
            padding_needed = next_power_of_2 - n
            current_data = np.pad(current_data, (0, padding_needed), 'constant')
            n = len(current_data)

        approx, detail = haar_dwt(current_data)
        coeffs.append(detail)  # Store detail coefficients
        current_data = approx  # Use approximation for the next level

    coeffs.append(current_data)  # Add the final approximation coefficients
    return coeffs[::-1] # Reverse to have [final_approximation, detail_level_N, ..., detail_level_1]


def multi_level_haar_idwt(coeffs):
    """
    Performs multi-level Inverse Discrete Wavelet Transform (IDWT) using the Haar wavelet.

    Args:
        coeffs (list): A list of arrays, where the first element is the final approximation
                       coefficients, and subsequent elements are the detail coefficients
                       from each level, starting from the deepest (as returned by multi_level_haar_dwt).

    Returns:
        np.ndarray: The reconstructed 1D data.
    """
    if not coeffs:
        return np.array([])

    # The last element in coeffs is the final approximation, the rest are details in reverse order of decomposition
    current_reconstruction = coeffs[0]

    for i in range(1, len(coeffs)):
        detail_coeffs = coeffs[i]
        # Pad detail_coeffs if their length doesn't match half of the current reconstruction length
        # This handles cases where original data was padded during DWT.
        if len(detail_coeffs) * 2 > len(current_reconstruction) * 2: # This condition might need refinement for edge cases.
             # More robust padding for IDWT:
             # The detail coefficients should always be half the length of the *source* approximation for that level.
             # If our `current_reconstruction` is larger due to previous padding, we need to ensure `detail_coeffs` matches.
             # For simplicity, assuming details match the decomposition structure.
             pass # In a perfect reconstruction scenario, detail_coeffs will naturally align.

        current_reconstruction = haar_idwt(current_reconstruction, detail_coeffs)

    return current_reconstruction


# --- Example Usage ---
if __name__ == "__main__":
    # Example 1: Single Level DWT and IDWT
    df = pd.read_csv('Data_August_Renewable.csv')
    data = df['Speed'].values
    # data1 = np.array([1, 2, 3, 4, 5, 6, 7, 8], dtype=float)
    print("--- Single Level Haar DWT and IDWT ---")
    print("Original Data 1:", data)

    approx_coeffs1, detail_coeffs1 = haar_dwt(data)
    print("Approximation Coeffs 1:", np.round(approx_coeffs1, 4))
    print("Detail Coeffs 1:", np.round(detail_coeffs1, 4))

    reconstructed_data1 = haar_idwt(approx_coeffs1, detail_coeffs1)
    print("Reconstructed Data 1:", np.round(reconstructed_data1, 4))
    print("Is Data 1 reconstructed perfectly?", np.allclose(data, reconstructed_data1))

    print("\n--- Single Level Haar DWT and IDWT (Odd Length) ---")
    data_odd = np.array([1, 2, 3, 4, 5], dtype=float)
    print("Original Data (Odd):", data_odd)
    approx_coeffs_odd, detail_coeffs_odd = haar_dwt(data_odd)
    print("Approximation Coeffs (Odd):", np.round(approx_coeffs_odd, 4))
    print("Detail Coeffs (Odd):", np.round(detail_coeffs_odd, 4))
    reconstructed_data_odd = haar_idwt(approx_coeffs_odd, detail_coeffs_odd)
    # Note: Reconstruction will be slightly off for the last element due to padding in DWT
    print("Reconstructed Data (Odd):", np.round(reconstructed_data_odd, 4))


    # Example 2: Multi-Level DWT and IDWT
    print("\n--- Multi-Level Haar DWT and IDWT ---")
    data2 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160], dtype=float)
    print("Original Data 2:", data2)
    levels = 3

    multi_coeffs = multi_level_haar_dwt(data2, levels)
    print(f"\nMulti-level DWT coefficients (Levels={levels}):")
    # multi_coeffs will be [final_approx, detail_L3, detail_L2, detail_L1]
    print("Final Approximation Coeffs:", np.round(multi_coeffs[0], 4))
    for i in range(1, len(multi_coeffs)):
        print(f"Detail Coeffs (Level {i}):", np.round(multi_coeffs[i], 4))

    reconstructed_data2 = multi_level_haar_idwt(multi_coeffs)
    # For multi-level, due to padding to nearest power of 2, the reconstructed data
    # might be longer than the original, and the padded part will be zeros.
    print("\nReconstructed Data 2:", np.round(reconstructed_data2, 4))
    # Compare only the original length
    print("Is Data 2 reconstructed perfectly (original length)?", np.allclose(data2, reconstructed_data2[:len(data2)]))

    print("\n--- Multi-Level Haar DWT and IDWT (Arbitrary Length) ---")
    data_arbitrary = np.array([1, 2, 3, 4, 5, 6, 7], dtype=float)
    print("Original Data (Arbitrary Length):", data_arbitrary)
    levels_arb = 2
    multi_coeffs_arb = multi_level_haar_dwt(data_arbitrary, levels_arb)
    print(f"\nMulti-level DWT coefficients (Levels={levels_arb}):")
    print("Final Approximation Coeffs:", np.round(multi_coeffs_arb[0], 4))
    for i in range(1, len(multi_coeffs_arb)):
        print(f"Detail Coeffs (Level {i}):", np.round(multi_coeffs_arb[i], 4))

    reconstructed_data_arb = multi_level_haar_idwt(multi_coeffs_arb)
    print("\nReconstructed Data (Arbitrary Length):", np.round(reconstructed_data_arb, 4))
    print("Is Data (Arbitrary Length) reconstructed perfectly (original length)?", np.allclose(data_arbitrary, reconstructed_data_arb[:len(data_arbitrary)]))

--- Single Level Haar DWT and IDWT ---
Original Data 1: [7.343 8.013 8.293 ... 9.517 9.46  9.309]
Approximation Coeffs 1: [10.8583 12.0512 12.9804 ... 13.0433 13.3954 13.2717]
Detail Coeffs 1: [-0.4738 -0.3231 -0.1662 ... -0.075  -0.0636  0.1068]
Reconstructed Data 1: [7.343 8.013 8.293 ... 9.517 9.46  9.309]
Is Data 1 reconstructed perfectly? True

--- Single Level Haar DWT and IDWT (Odd Length) ---
Original Data (Odd): [1. 2. 3. 4. 5.]
Approximation Coeffs (Odd): [2.1213 4.9497 3.5355]
Detail Coeffs (Odd): [-0.7071 -0.7071  3.5355]
Reconstructed Data (Odd): [1. 2. 3. 4. 5. 0.]

--- Multi-Level Haar DWT and IDWT ---
Original Data 2: [ 10.  20.  30.  40.  50.  60.  70.  80.  90. 100. 110. 120. 130. 140.
 150. 160.]

Multi-level DWT coefficients (Levels=3):
Final Approximation Coeffs: [127.2792 353.5534]
Detail Coeffs (Level 1): [-56.5685 -56.5685]
Detail Coeffs (Level 2): [-20. -20. -20. -20.]
Detail Coeffs (Level 3): [-7.0711 -7.0711 -7.0711 -7.0711 -7.0711 -7.0711 -7.0711 -7.0711]

R